In [1]:
import sys
import numpy as np
from PNM import *
# define pi for convenience
pi = np.pi

def CreateAndSavePFM(out_path):
    width = 512
    height = 512
    numComponents = 3

    img_out = np.empty(shape=(width, height, numComponents), dtype=np.float32)
    
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = 1.0

    write_pfm(out_path, img_out)

def LoadAndSavePPM(in_path, out_path):
    img_in = read_ppm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=img_in.dtype)
    height,width,_ = img_in.shape # Retrieve height and width
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] # Copy pixels

    write_ppm(out_path, img_out)

def LoadAndSavePFM(in_path, out_path):
    img_in = read_pfm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=img_in.dtype)
    height,width,_ = img_in.shape # Retrieve height and width
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] # Copy pixels

    write_pfm(out_path, img_out)

def LoadPPMAndSavePFM(in_path, out_path):
    img_in = read_ppm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=np.float32)
    height,width,_ = img_in.shape
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:]/255.0

    write_pfm(out_path, img_out)
            
def LoadPFMAndSavePPM(in_path, out_path):
    img_in = read_pfm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=np.float32)
    height,width,_ = img_in.shape
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] * 255.0

    write_ppm(out_path, img_out.astype(np.uint8))

def create_sphere_image():
    width = 511
    height = 511
    numComponents = 3

    img_out = np.empty(shape=(width, height, numComponents), dtype=np.float32)
    # the sphere has diameter 511, the center point is the center of the image, the other part outside the sphere is black
    for y in range(height):
        for x in range(width):
            if (x - 255.5) ** 2 + (y - 255.5) ** 2 <= 255.5 ** 2:
                img_out[y, x, :] = 1.0
            else:
                img_out[y, x, :] = 0.0

    return img_out


In [11]:
environment_graph = read_pfm('../Office/Office6.pfm')
sphere = create_sphere_image()
print(sphere.shape)
print(environment_graph.shape)

(511, 511, 3)
(576, 864, 3)


In [10]:
# Cartesian to polar coordinates conversion
def cart2pol(x, y):
    theta = np.arctan2(y, x)
    rho = np.sqrt(x ** 2 + y ** 2)
    return theta, rho

# transform the environment map to a sphere
def environment_to_sphere(environment_graph):
    width, height, _ = environment_graph.shape
    sphere = np.empty(shape=(width, height, 3), dtype=np.float32)
    for y in range(height):
        for x in range(width):
            theta, rho = cart2pol(x - 255.5, y - 255.5)
            if rho <= 255.5:
                # calculate the coordinates of the corresponding point in the environment map
                x_env = int((theta + pi) / (2 * pi) * width)
                y_env = int((pi / 2 - np.arcsin(rho / 255.5)) / pi * height)
                sphere[y, x, :] = environment_graph[y_env, x_env, :]
            else:
                print(y,x)
                sphere[y, x, :] = 0.0
    return sphere

print('Converting environment map to sphere...')
sphere_transfered = environment_to_sphere(environment_graph)
write_pfm('../Office6_sphere.pfm', sphere)

Converting environment map to sphere...
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 

IndexError: index 576 is out of bounds for axis 0 with size 576

6